**ROW_NUMBER()** : atribui um número inteiro sequencial e exclusivo a cada linha de um conjunto de resultados, começando em 1, útil para classificar, paginar ou identificar linhas específicas, especialmente com OVER (PARTITION BY ... ORDER BY ...) para definir grupos e ordens dentro desses grupos.

**Exemplos de uso**
- **Rankear produtos:** ROW_NUMBER() OVER (ORDER BY preco DESC) para numerar produtos do mais caro para o mais barato.
- **Paginagem:** Encontrar a "página" de resultados em consultas maiores.
- **Identificar o "top N" por grupo**: ROW_NUMBER() OVER (PARTITION BY categoria ORDER BY data_venda DESC) para encontrar a venda mais recente em cada categoria. 

In [0]:
-- Ranking de pedidos por valor
WITH pedidos AS (
  SELECT
    order_id,
    SUM(price) AS valor_pedido
  FROM workspace.projeto_vendas_bronze.order_items
  GROUP BY order_id
)

SELECT
  order_id,
  valor_pedido,
  ROW_NUMBER() OVER (ORDER BY valor_pedido DESC) AS ranking
FROM pedidos;

In [0]:
-- Pedido mais caro por Estado
WITH pedidos AS (
  SELECT
    o.order_id,
    c.customer_state,
    SUM(oi.price) AS valor_pedido
  FROM workspace.projeto_vendas_bronze.order_items oi
  JOIN workspace.projeto_vendas_bronze.orders o
    ON oi.order_id = o.order_id
  JOIN workspace.projeto_vendas_bronze.customers c
    ON oi.order_id = o.order_id
  GROUP BY o.order_id, c.customer_state
)

SELECT * FROM (
  SELECT
    order_id,
    customer_state,
    valor_pedido,
    ROW_NUMBER() OVER (PARTITION BY customer_state ORDER BY valor_pedido DESC) AS rn
  FROM pedidos
) t
WHERE rn = 1

**RANK():** atribui uma classificação (posição) a cada linha dentro de um conjunto de resultados, com base em uma ordenação específica definida pela cláusula OVER (ORDER BY...), sendo ideal para criar tabelas de classificação, como os melhores vendedores ou pontuações, onde empates resultam na mesma classificação e as posições seguintes são puladas (ex: 1, 2, 2, 4).

In [0]:
-- Ranking de categorias por faturamento
WITH categorias AS (
  SELECT
    p.product_category_name,
    SUM(oi.price) AS faturamento
  FROM workspace.projeto_vendas_bronze.order_items oi
  JOIN workspace.projeto_vendas_bronze.products p
    ON oi.product_id = p.product_id
  GROUP BY p.product_category_name
)

SELECT
  product_category_name,
  faturamento,
  RANK() OVER (ORDER BY faturamento DESC) AS ranking_categorias
FROM categorias

**PARTITION BY**:

In [0]:
-- Top 3 categorias por estado
WITH vendas AS (
  SELECT
    p.product_category_name,
    c.customer_state,
    SUM(oi.price) AS faturamento
  FROM workspace.projeto_vendas_bronze.order_items oi
  JOIN workspace.projeto_vendas_bronze.orders o
    ON oi.order_id = o.order_id
  JOIN workspace.projeto_vendas_bronze.customers c
    ON o.customer_id = c.customer_id
  JOIN workspace.projeto_vendas_bronze.products p
    ON oi.product_id = p.product_id
  GROUP BY p.product_category_name, c.customer_state
)
SELECT * FROM (
  SELECT
    product_category_name,
    customer_state,
    faturamento,
    DENSE_RANK() OVER (PARTITION BY customer_state ORDER BY faturamento DESC) AS ranking
  FROM vendas
) t
WHERE ranking <= 3;

**OVER() sem ORDER BY**

In [0]:
-- Participação percentual no faturamento
WITH categoria AS (
  SELECT
    p.product_category_name,
    SUM(oi.price) AS faturamento
  FROM workspace.projeto_vendas_bronze.order_items oi
  JOIN workspace.projeto_vendas_bronze.products p
    ON oi.product_id = p.product_id
  GROUP BY p.product_category_name
)

SELECT
  product_category_name,
  faturamento,
  faturamento / SUM(faturamento) OVER() AS participacao_percentual
FROM categoria
ORDER BY participacao_percentual DESC;